# Preprare configs

This notebook supports the process of preparing config files.
The goals of the notebooks are as follows:
- extracting selected frame(s) from the video
- setting ant testing unwrapping parameters (camera view -> bird's eye view)

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def dispimges(im1, im2, BGR2RGB1=True, BGR2RGB2=True, im1_title='Original Image', im2_title = 'Processed image', fontsize=20):
    iscolor1 = len(im1.shape) > 2
    iscolor2 = len(im2.shape) > 2 
    if BGR2RGB1 and iscolor1:
        im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)
    if BGR2RGB2 and iscolor2:
        im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

    if iscolor1:
        ax1.imshow(im1)
    else:
        ax1.imshow(im1, cmap='gray')
    ax1.set_title(im1_title, fontsize=fontsize)
    if iscolor2:
        ax2.imshow(im2)
    else:
        ax2.imshow(im2, cmap='gray')
    ax2.set_title(im2_title, fontsize=fontsize)
    return ax1, ax2

def extract_frame(video_path = None, cap = None, frameno = 1, dest_file = None, display = True):
    """
    Extracts (and optionally saves) one frame from the video
    Arguments:
        video_path - path to the video
        cap        - captured video (results of cv2.VideoCapture). Works only if video_path is None
        frameno    - number of frame to be extracted
        dest_file  - where should the frame be saved
        display    - it True, the frame is displayed        
    """
    if video_path is None:
        if cap is None:
             raise Exception('Either video_path or cap must be set.')
    else:
        cap = cv2.VideoCapture(video_path)
        
    cap.set(1,frameno);
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    ret, image = cap.read()
    if dest_file is not None:
        cv2.imwrite(dest_file,image)
    
    if display:
        im2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        f = plt.figure(figsize=(20, 10))
        plt.imshow(im2)
    return image

In [ ]:
VIDEO_FILE = '/data/MoveOver/Videos/I97-New Cut Road HH 3-4-21.mp4'
extract_frame(VIDEO_FILE, frameno = 100, dest_file = './frame_100.jpg');
extract_frame(VIDEO_FILE, frameno = 100, dest_file = './frame_115.jpg');


Points for source (camera view) and destination (image view)
First four points are used for unwrapping, the rest is just display at tests

In [ ]:
SRC = np.float32([
        [446, 1016],
        [1231, 970],
        [842, 759],
        [1377, 787],
        [1023, 299],
    ])
    
DST = np.float32([
        [110, 37],
        [79, 104],
        [174, 220],
        [127, 236],
        [620, 1177],
    ])
    

In [ ]:
M = cv2.getPerspectiveTransform(SRC[:4], DST[:4])
Minv = cv2.getPerspectiveTransform(DST[:4], SRC[:4])


In [ ]:
def display_point(points, M, img1_path, img2_path):
    src = np.array([points], dtype='float32')

    print (src)
    trans = cv2.perspectiveTransform(src, M)
    
    img = cv2.imread(img1_path)
    mapimg = cv2.imread(img2_path)
    
    ax1, ax2 = dispimges(img, mapimg)

    if True: #show points
        for coor in src[0]:
            ax1.plot(coor[0], coor[1], '.', color='red')
        for coor in trans[0]:
            ax2.plot(coor[0], coor[1], '.', color='red')
    return trans

In [ ]:
ret = display_point(SRC, M, './frame_100.jpg', 'I97_NewCutRoad.jpg')

In [ ]:
ret = display_point(DST, Minv, 'I97_NewCutRoad.jpg',  './frame_100.jpg')

The code below displays the parameters in the format that can be pasted directly to config.json

In [ ]:
print('  "cameraPoints" : [[{}, {}], [{}, {}], [{}, {}], [{}, {}]],'.format(
    SRC[0][0], SRC[0][1], SRC[1][0], SRC[1][1], SRC[2][0], SRC[2][1],SRC[3][0], SRC[3][1]))

print('  "birdEyePoints" : [[{}, {}], [{}, {}], [{}, {}], [{}, {}]],'.format(
    DST[0][0], DST[0][1], DST[1][0], DST[1][1], DST[2][0], DST[2][1], DST[3][0], DST[3][1])) 

# Lane masks

In [ ]:
MASK_PATH = '../images/lane_masks/I97_NewCutRoad_Mask.png'

In [ ]:


mask = (255*plt.imread(MASK_PATH)).astype(int)
if (len(mask.shape) == 3) and (mask.shape[2] > 1):
    #mask = np.sum(mask, axis=2)
    mask = mask[:, :, 0]
plt.imshow(mask, cmap='gray')
list(np.unique(mask))

In [ ]:
plt.imshow(mask, cmap='gray')